<h1>Imports</h1>

<table class="MsoTableGrid" border="1" cellspacing="0" cellpadding="0" width="1589" style="width:1192.0pt;border-collapse:collapse;border:none;mso-border-alt:
 solid windowtext .5pt;mso-yfti-tbllook:1184;mso-padding-alt:0in 5.4pt 0in 5.4pt">
 <tbody><tr style="mso-yfti-irow:0;mso-yfti-firstrow:yes">
  <td width="397" valign="top" style="width:298.0pt;border:solid windowtext 1.0pt;
  mso-border-alt:solid windowtext .5pt;background:#D9D9D9;mso-background-themecolor:
  background1;mso-background-themeshade:217;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center"><span style="color:black;mso-color-alt:windowtext">Project</span></p>
  </td>
  <td width="397" valign="top" style="width:298.0pt;border:solid windowtext 1.0pt;
  border-left:none;mso-border-left-alt:solid windowtext .5pt;mso-border-alt:
  solid windowtext .5pt;background:#D9D9D9;mso-background-themecolor:background1;
  mso-background-themeshade:217;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center"><span style="color:black;mso-color-alt:windowtext">Developer</span></p>
  </td>
  <td width="397" valign="top" style="width:298.0pt;border:solid windowtext 1.0pt;
  border-left:none;mso-border-left-alt:solid windowtext .5pt;mso-border-alt:
  solid windowtext .5pt;background:#D9D9D9;mso-background-themecolor:background1;
  mso-background-themeshade:217;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center"><span style="color:black;mso-color-alt:windowtext">Tools</span></p>
  </td>
  <td width="397" valign="top" style="width:298.0pt;border:solid windowtext 1.0pt;
  border-left:none;mso-border-left-alt:solid windowtext .5pt;mso-border-alt:
  solid windowtext .5pt;background:#D9D9D9;mso-background-themecolor:background1;
  mso-background-themeshade:217;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center"><span style="color:black;mso-color-alt:windowtext">Version</span></p>
  </td>
 </tr>
 <tr style="mso-yfti-irow:1">
  <td width="397" valign="top" style="width:298.0pt;border:solid windowtext 1.0pt;
  border-top:none;mso-border-top-alt:solid windowtext .5pt;mso-border-alt:solid windowtext .5pt;
  padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center">Bootcamp Project 4</p>
  </td>
  <td width="397" valign="top" style="width:298.0pt;border-top:none;border-left:
  none;border-bottom:solid windowtext 1.0pt;border-right:solid windowtext 1.0pt;
  mso-border-top-alt:solid windowtext .5pt;mso-border-left-alt:solid windowtext .5pt;
  mso-border-alt:solid windowtext .5pt;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center">Patrick Ryan</p>
  </td>
  <td width="397" valign="top" style="width:298.0pt;border-top:none;border-left:
  none;border-bottom:solid windowtext 1.0pt;border-right:solid windowtext 1.0pt;
  mso-border-top-alt:solid windowtext .5pt;mso-border-left-alt:solid windowtext .5pt;
  mso-border-alt:solid windowtext .5pt;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center">Python 3.12</p>
  </td>
  <td width="397" valign="top" style="width:298.0pt;border-top:none;border-left:
  none;border-bottom:solid windowtext 1.0pt;border-right:solid windowtext 1.0pt;
  mso-border-top-alt:solid windowtext .5pt;mso-border-left-alt:solid windowtext .5pt;
  mso-border-alt:solid windowtext .5pt;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center">1</p>
  </td>
 </tr>
 <tr style="mso-yfti-irow:2">
  <td width="1589" colspan="4" valign="top" style="width:1192.0pt;border:solid windowtext 1.0pt;
  border-top:none;mso-border-top-alt:solid windowtext .5pt;mso-border-alt:solid windowtext .5pt;
  background:#D9D9D9;mso-background-themecolor:background1;mso-background-themeshade:
  217;padding:0in 5.4pt 0in 5.4pt">
  <p class="MsoNormal" align="center" style="text-align:center"><span style="color:black;mso-color-alt:windowtext">Description</span></p>
  </td>
 </tr>
 <tr style="mso-yfti-irow:3;mso-yfti-lastrow:yes">
  <td width="1589" colspan="4" valign="top" style="width:1192.0pt;border:solid windowtext 1.0pt;
  border-top:none;mso-border-top-alt:solid windowtext .5pt;mso-border-alt:solid windowtext .5pt;
  padding:0in 5.4pt 0in 5.4pt">
  <h1>Libraries and Modules:<o:p></o:p></h1>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l11 level1 lfo9;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Data
  Handling and Computation Libraries:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->pandas: For data manipulation.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">numpy</span>: For numerical
  operations.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->math: For mathematical functions.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l11 level1 lfo9;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Machine
  Learning and Data Preprocessing Libraries:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->scikit-learn: For machine learning
  functionalities, including model selection, preprocessing, and feature
  selection.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">tensorflow</span>: For
  building and training deep learning models.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l11 level1 lfo9;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Visualization
  Libraries:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE"><span class="GramE">matplotlib.pyplot</span></span>
  and seaborn: For data visualization.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l11 level1 lfo9;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Deep
  Learning and Neural Network Libraries:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE"><span class="GramE">tensorflow.keras</span></span>:
  For building and training neural network models.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l11 level2 lfo9;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">keras_tuner</span>: For
  hyperparameter tuning in <span class="SpellE">Keras</span> models.<o:p></o:p></p>
  <h1>Functions:<o:p></o:p></h1>
  <p class="MsoListParagraphCxSpFirst" style="text-indent:-.25in;mso-list:l1 level1 lfo14"><!--[if !supportLists]--><span style="mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin"><span style="mso-list:Ignore">1.<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">load_data</span>: To load
  data into a <span class="SpellE">DataFrame</span>.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="text-indent:-.25in;mso-list:l1 level1 lfo14"><!--[if !supportLists]--><span style="mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin"><span style="mso-list:Ignore">2.<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">identify_data_types</span>:
  To categorize columns into numerical and categorical based on unique values.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="text-indent:-.25in;mso-list:l1 level1 lfo14"><!--[if !supportLists]--><span style="mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin"><span style="mso-list:Ignore">3.<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">exploratory_data_analysis</span>:
  For conducting exploratory data analysis.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="text-indent:-.25in;mso-list:l1 level1 lfo14"><!--[if !supportLists]--><span style="mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin"><span style="mso-list:Ignore">4.<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">normalize_and_encode</span>:
  To normalize numerical columns and encode categorical columns.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="text-indent:-.25in;mso-list:l1 level1 lfo14"><!--[if !supportLists]--><span style="mso-bidi-font-family:Calibri;mso-bidi-theme-font:minor-latin"><span style="mso-list:Ignore">5.<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">handle_missing_values</span>:
  To handle missing values in the dataset.<o:p></o:p></p>
  <h1>Main Workflow:<o:p></o:p></h1>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Setup:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Configuration of logging and handling of
  TensorFlow logging levels.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Data
  Loading:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Reading of data from a specified source using
  the <span class="SpellE">load_data</span> function.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Exploratory
  Data Analysis (EDA):<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Conducted using the <span class="SpellE">exploratory_data_analysis</span>
  function.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Involves analysis of both numerical and
  categorical features in the dataset.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Target variable analysis focused on the 'ICU'
  column.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Data
  Preprocessing:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Normalization of numerical columns and
  encoding of categorical columns using the <span class="SpellE">normalize_and_encode</span>
  function.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Handling of missing values in the dataset
  through the <span class="SpellE">handle_missing_values</span> function.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Model
  Building:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->The script includes processes for constructing
  machine learning models.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Utilizes TensorFlow and <span class="SpellE">Keras</span>
  for creating and training deep learning models.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Incorporates hyperparameter tuning using <span class="SpellE">keras_tuner</span>.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Model
  Evaluation:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Likely includes functions and steps to
  evaluate the performance of the machine learning models.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpMiddle" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Metrics such as accuracy, precision, recall,
  F1-score, and ROC-AUC are probably computed.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Evaluation may also involve confusion
  matrices, ROC curves, and other visualizations to assess model performance.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Feature
  Selection:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Processes for feature selection or reduction
  to improve model performance, potentially using methods like PCA, RFE, or
  others.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Hyperparameter
  Tuning:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]--><span class="SpellE">keras_tuner</span> is used
  for hyperparameter tuning of neural network models, aiming to find the best
  model performance.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Additional
  Functionalities:<o:p></o:p></h2>
  <p class="MsoListParagraphCxSpFirst" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->May include data augmentation, advanced
  preprocessing techniques, or custom loss functions for deep learning models.<o:p></o:p></p>
  <p class="MsoListParagraphCxSpLast" style="margin-left:1.0in;mso-add-space:
  auto;text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Logging and debugging practices to track the
  script's execution and performance.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Custom
  Functions:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Besides the ones mentioned, there might be
  additional utility functions for data transformation, feature engineering, or
  model diagnostics.<o:p></o:p></p>
  <h2 style="margin-left:.5in;text-indent:-.25in;mso-list:l2 level1 lfo15;
  tab-stops:list .5in"><!--[if !supportLists]--><span style="font-size:10.0pt;
  mso-bidi-font-size:13.0pt;font-family:Symbol;mso-fareast-font-family:Symbol;
  mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; </span></span></span><!--[endif]-->Utilities
  and Helpers:<o:p></o:p></h2>
  <p class="MsoListParagraph" style="margin-left:1.0in;mso-add-space:auto;
  text-indent:-.25in;mso-list:l2 level2 lfo15;tab-stops:list 1.0in"><!--[if !supportLists]--><span style="font-size:10.0pt;mso-bidi-font-size:12.0pt;font-family:Symbol;
  mso-fareast-font-family:Symbol;mso-bidi-font-family:Symbol"><span style="mso-list:Ignore">·<span style="font:7.0pt &quot;Times New Roman&quot;">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
  </span></span></span><!--[endif]-->Helper classes or utilities for tasks like
  data loading, data splitting, or handling specific data types, such as
  time-series.<o:p></o:p></p>
  <p class="MsoNormal" style="margin-left:.5in"><o:p>&nbsp;</o:p></p>
  </td>
 </tr>
</tbody></table>

In [ ]:
# Basic data manipulation and numerical operations
import pandas as pd  # For data manipulation and analysis
import numpy as np  # For numerical operations

# Logging and utilities
import logging  # For logging information during execution
import math  # For basic mathematical operations

# Data visualization
import matplotlib.pyplot as plt  # For plotting graphs and charts
import seaborn as sns  # For making statistical graphics

# Machine Learning - Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder  # For feature scaling and encoding categorical variables
from sklearn.compose import ColumnTransformer  # For applying transformers to columns of Pandas dataframes

# Machine Learning - Model Selection and Validation
from sklearn.model_selection import train_test_split  # For splitting data into training and testing sets
from sklearn.model_selection import KFold  # For K-Folds cross-validator

# Machine Learning - Metrics
from sklearn.metrics import classification_report, accuracy_score  # For evaluating model performance

# Machine Learning - Models
from sklearn.ensemble import RandomForestClassifier  # For the Random Forest algorithm
from sklearn.feature_selection import SelectFromModel  # For feature selection based on importance weights

# Deep Learning with TensorFlow and Keras
import tensorflow as tf  # For deep learning models
from tensorflow.keras.models import Sequential  # For linear stack of layers in neural networks
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten, Dropout  # Different types of neural network layers
from tensorflow.keras.optimizers import Adam  # Optimizer for training neural networks
from tensorflow.keras import regularizers  # Regularizers to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler  # Callbacks for better control over model training
from keras_tuner.tuners import Hyperband # Tuner for hyperparameter optimization
from keras_tuner.engine.hyperparameters import HyperParameters # To manage hyperparameters for tuning

<h1>Logging</h1>

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logging.getLogger('tensorflow').setLevel(logging.ERROR)

<h1>Function Declarations</h1>

In [ ]:
def load_data(data):
    """
    Load data from a CSV file into a pandas DataFrame.

    This function attempts to read a CSV file specified by the 'data' parameter. 
    It includes error handling to manage situations where the file is not found, 
    the data is empty, or there's a parsing error.

    Parameters:
    data (str): The file path of the CSV file to be loaded.

    Returns:
    pd.DataFrame: A DataFrame containing the data from the CSV file.

    Raises:
    FileNotFoundError: If the CSV file is not found at the specified path.
    pd.errors.EmptyDataError: If the CSV file is empty.
    pd.errors.ParserError: If there is an error parsing the CSV file.
    """

    try:
        df = pd.read_csv(data)  # Attempt to read the CSV file into a pandas DataFrame
    except (FileNotFoundError, pd.errors.EmptyDataError, pd.errors.ParserError) as e:
        logging.error(f"Error: {str(e)}.")  # Log the error
        exit(1)  # Exit the program if an error occurs
    return df

In [ ]:
def identify_data_types(df, unique_value_threshold=10):
    """
    Identify the data types of columns in a DataFrame as either numerical or categorical.

    This function iterates through each column of the DataFrame. It classifies columns 
    as 'categorical' if they are of object type or if the number of unique values 
    is less than or equal to the specified threshold. All other columns are classified as 'numerical'.

    Parameters:
    df (pd.DataFrame): The DataFrame whose columns are to be classified.
    unique_value_threshold (int, optional): The threshold for determining if a column with numerical 
                                            values should be treated as categorical. Defaults to 10.

    Returns:
    tuple: A tuple containing two lists: the first list contains the names of numerical columns, 
           and the second list contains the names of categorical columns.
    """
    numerical_columns = []  # Initialize a list to store names of numerical columns
    categorical_columns = []  # Initialize a list to store names of categorical columns

    for column in df.columns:
        if df[column].dtype == 'object':
            # Add column to categorical list if it's of object type
            categorical_columns.append(column)
        else:
            # If the number of unique values is less than or equal to the threshold, consider it categorical
            if df[column].nunique() <= unique_value_threshold:
                categorical_columns.append(column)
            else:
                # Otherwise, consider it a numerical column
                numerical_columns.append(column)
                
    return numerical_columns, categorical_columns

In [ ]:
def exploratory_data_analysis(dataframe, numerical_columns, categorical_columns, target_column='ICU', num_features=16):
    """
    Perform exploratory data analysis (EDA) on a given DataFrame.

    This function conducts a series of analyses, including feature correlation analysis, descriptive statistics,
    missing values analysis, and distribution analysis. It focuses on both numerical and categorical features,
    particularly highlighting those most correlated with a specified target column. The function visualizes
    the distribution of the top numerical features and the percentage of missing values in them.

    Parameters:
    dataframe (pd.DataFrame): The DataFrame to analyze.
    numerical_columns (list): A list of names of numerical columns in the DataFrame.
    categorical_columns (list): A list of names of categorical columns in the DataFrame.
    target_column (str, optional): The name of the target column for correlation analysis. Defaults to 'ICU'.
    num_features (int, optional): The number of top features to consider for analysis. Defaults to 16.

    Outputs:
    - Prints the top features correlated with the target column.
    - Prints descriptive statistics for the top numerical columns.
    - Prints an analysis of missing values for the top features.
    - Displays bar plots for missing values percentage and histograms for the distribution of top numerical features.

    Returns:
    None: This function does not return any value.
    """
    # Feature Correlation Analysis: Determine the correlation of each numerical feature with the target column
    correlation = dataframe[numerical_columns].corrwith(dataframe[target_column]).abs()
    top_features = correlation.sort_values(ascending=False).head(num_features).index.tolist()
    print(f"Top {num_features} features correlated with {target_column}: {top_features}")

    # Descriptive Statistics: Provide summary statistics for the top correlated numerical features
    print("\nDescriptive Statistics for Top Numerical Columns:\n", dataframe[top_features].describe())

    # Missing Values Analysis: Calculate and display the number and percentage of missing values in the top features
    missing_values = dataframe[top_features].isna().sum()
    missing_percentage = (missing_values / len(dataframe)) * 100
    missing_df = pd.DataFrame({'Number of Missing Values': missing_values, 'Percentage': missing_percentage})
    print("\nMissing Values Analysis for Top Features:\n", missing_df)

    # Visualization of Missing Values: Create a bar plot to visualize the percentage of missing values in top features
    plt.figure(figsize=(12, 6))
    sns.barplot(x=missing_df.index, y=missing_df['Percentage'])
    plt.xticks(rotation=90)  # Rotating the feature names
    plt.title("Missing Values Percentage for Top Features")
    plt.xlabel("Features")
    plt.ylabel("Percentage of Missing Values")
    plt.show()

    # Distribution Analysis for top numerical columns: Plot histograms to visualize the distribution of top numerical features
    top_numerical_features = [col for col in top_features if col in numerical_columns]
    if top_numerical_features:
        # Determine the size of the grid for plotting histograms
        grid_size_num = int(math.ceil(math.sqrt(len(top_numerical_features))))
        fig, axes = plt.subplots(grid_size_num, grid_size_num, figsize=(15, 15))
        axes = axes.flatten()
        
        # Plotting histogram for each numerical feature
        for i, col in enumerate(top_numerical_features):
            sns.histplot(dataframe[col], kde=True, bins=30, ax=axes[i])
            axes[i].set_title(f"Distribution of {col}", fontsize=10)  # Title for each subplot
            axes[i].set_xlabel('')  # Clearing x-labels for neatness
            axes[i].set_ylabel('')  # Clearing y-labels for neatness

        # Removing any extra subplots that are not used
        for ax in axes[i+1:]:
            ax.remove()

        plt.tight_layout()
        plt.show()

In [ ]:
def normalize_and_encode(df, numerical_cols, categorical_cols):
    """
    Normalize numerical columns and encode categorical columns in a DataFrame.

    This function applies standard scaling to numerical columns and one-hot encoding to categorical columns.
    It combines these transformed columns into a single DataFrame, which is suitable for machine learning models.

    Parameters:
    df (pd.DataFrame): The DataFrame to be transformed.
    numerical_cols (list): A list of names of numerical columns to be normalized.
    categorical_cols (list): A list of names of categorical columns to be encoded.

    Returns:
    pd.DataFrame: A DataFrame with normalized numerical columns and one-hot encoded categorical columns.
    """

    # Define the transformations for numerical and categorical columns
    numerical_transformer = StandardScaler()  # StandardScaler for normalizing numerical data
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)  # OneHotEncoder for encoding categorical data

    # Create the column transformer with specified transformers
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Fit the transformer to the data and then transform it
    df_transformed = preprocessor.fit_transform(df)

    # Get feature names after one-hot encoding
    one_hot_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
    all_feature_names = numerical_cols + list(one_hot_feature_names) # Combine numerical and encoded categorical feature names
    
    # Create a new DataFrame with the transformed data
    df_transformed = pd.DataFrame(df_transformed, columns=all_feature_names)  # Assign column names to the transformed DataFrame

    return df_transformed  # Return the transformed DataFrame

In [ ]:
def handle_missing_values(df):
    """
    Handle missing values in a DataFrame.

    This function creates indicators for missing data, applies forward fill imputation, and combines
    the imputed data with the missingness indicators. Forward fill is used first, followed by backward fill
    to address any remaining missing values.

    Parameters:
    df (pd.DataFrame): The DataFrame with missing values to handle.

    Returns:
    pd.DataFrame: A DataFrame where missing values have been imputed, and missingness indicators have been added.
    """

    # Create missingness indicators for each column
    missing_indicators = df.isna().astype(int).add_suffix('_missing')  # 1 indicates missing, 0 otherwise

    # Apply forward fill imputation (filling the missing value with the previous value in the column)
    df_imputed = df.fillna(method='ffill')  # Forward fill
    df_imputed.fillna(method='bfill', inplace=True)  # Backward fill for any remaining missing values

    # Combine imputed data with missingness indicators
    df_combined = pd.concat([df_imputed, missing_indicators], axis=1)  # Add indicators to the DataFrame

    return df_combined  # Return the combined DataFrame with imputed values and indicators

In [ ]:
def add_missing_columns(df_no_missing, df):
    """
    Add missing columns 'PATIENT_VISIT_IDENTIFIER', 'ICU', and 'WINDOW' to the DataFrame if they are not present.

    Args:
        df_no_missing (pd.DataFrame): The DataFrame without missing columns.
        df (pd.DataFrame): The original DataFrame containing the missing columns.

    Returns:
        pd.DataFrame: The updated DataFrame with missing columns added.
    """
    # Check if 'PATIENT_VISIT_IDENTIFIER' is missing and add it if needed
    if 'PATIENT_VISIT_IDENTIFIER' not in df_no_missing.columns:
        df_no_missing['PATIENT_VISIT_IDENTIFIER'] = df['PATIENT_VISIT_IDENTIFIER']
    
    # Check if 'ICU' is missing and add it if needed
    if 'ICU' not in df_no_missing.columns:
        df_no_missing['ICU'] = df['ICU']
    
    # Check if 'WINDOW' is missing and add it if needed
    if 'WINDOW' not in df_no_missing.columns:
        df_no_missing['WINDOW'] = df['WINDOW']
    
    return df_no_missing

In [ ]:
def feature_selection(df, target_column='ICU', threshold=0.007, additional_cols=['PATIENT_VISIT_IDENTIFIER', 'WINDOW']):
    """
    Perform feature selection on a DataFrame based on feature importance using a RandomForestClassifier.

    This function handles missing values, separates the target variable and additional specified columns,
    encodes categorical variables, and uses a RandomForestClassifier to determine feature importance. 
    Features above a specified importance threshold are selected.

    Parameters:
    df (pd.DataFrame): The DataFrame to perform feature selection on.
    target_column (str, optional): The target variable for the classification. Defaults to 'ICU'.
    threshold (float, optional): The threshold for feature importance. Defaults to 0.007.
    additional_cols (list, optional): Additional columns to retain in the final DataFrame. Defaults to ['PATIENT_VISIT_IDENTIFIER', 'WINDOW'].

    Returns:
    tuple: A tuple containing the DataFrame with selected features and a list of the selected feature names.
    """

    # Handling missing values - consider modifying this as per your dataset's requirements
    df = df.dropna()  # Dropping rows with missing values

    # Separating the target variable
    y = df[target_column]  # Extract the target column

    # Separating the additional columns to retain them later
    additional_data = df[additional_cols]  # Extract additional columns

    # Dropping the target and additional columns from the main DataFrame
    df = df.drop(additional_cols + [target_column], axis=1, errors='ignore')  # Remove non-feature columns

    # Encoding categorical variables if any
    df_encoded = pd.get_dummies(df, drop_first=True)  # One-hot encode categorical variables

    # Using RandomForestClassifier for feature importance
    model = RandomForestClassifier()
    model.fit(df_encoded, y)  # Train the model

    # Selecting features based on importance
    sfm = SelectFromModel(model, threshold=threshold) # Initialize the feature selector with the model
    sfm.fit(df_encoded, y) # Fit the feature selector to the data
    
    # Getting the names of selected features
    feature_names = df_encoded.columns[sfm.get_support()]  # Extract the names of important features

    # Displaying selected features and their count
    num_selected_features = len(feature_names)
    print(f"Number of Selected Features: {num_selected_features}")
    print("Selected Features:")
    for feature in feature_names:
        print(feature)

    # Creating a DataFrame with selected features and additional columns
    df_selected = df_encoded[feature_names].join(additional_data).join(y)  # Combine selected features with additional data and the target column

    return df_selected, feature_names  # Return the DataFrame with selected features and the list of feature names

In [ ]:
def prepare_sequence_data(df, patient_identifier_col, target_col, window_col):
    """
    Prepare sequence data from a DataFrame for sequence-based models.

    This function sorts the data by patient identifier and time window, groups by patient identifier, and then
    creates sequences of data for each patient. It accounts for the first instance of ICU admission and 
    excludes data after this instance. The function is particularly useful for time-series or sequence prediction tasks.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data.
    patient_identifier_col (str): Column name representing the patient identifier.
    target_col (str): Column name representing the target variable (e.g., ICU admission status).
    window_col (str): Column name representing the time window or sequence order.

    Returns:
    tuple: A tuple containing two elements, a list of sequences (X) and a list of labels (y).
    """

    # Sort the DataFrame by patient identifier and time window for sequence generation
    df_sorted = df.sort_values(by=[patient_identifier_col, window_col])

    # Group the data by patient identifier
    grouped = df_sorted.groupby(patient_identifier_col)
    
    X = []  # Initialize an empty list to store feature sequences
    y = []  # Initialize an empty list to store labels (target variable values)

    for _, group in grouped:
        # Determine the first instance of ICU admission in each group
        first_icu_admission = group[target_col].cumsum().shift(fill_value=0).eq(1)

        # Exclude data points after the first ICU admission for each patient
        group = group[~first_icu_admission]
        
        # Drop columns that are not features, such as identifiers and the target column
        features = group.drop(columns=[patient_identifier_col, target_col, window_col])

        # Append the sequence of features for each patient to the list X
        X.append(features.values)

        # Append the target label for each sequence to the list y
        # Using max() ensures capturing whether the patient was ever admitted to the ICU within the window
        y.append(group[target_col].max())

    return X, y  # Return the lists of feature sequences and corresponding labels

In [ ]:
def create_lstm_cnn_model(input_shape, num_classes, filters, kernel_size, lstm_units, dense_units, dropout_rate, learning_rate):
    """
    Create a Convolutional Neural Network (CNN) followed by a Long Short-Term Memory (LSTM) model.

    Args:
        input_shape (tuple): The shape of the input data (e.g., (sequence_length, input_features)).
        num_classes (int): The number of output classes.
        filters (int): The number of filters in the convolutional layer.
        kernel_size (int): The size of the convolutional kernel.
        lstm_units (int): The number of units in the LSTM layer.
        dense_units (int): The number of units in the fully connected dense layer.
        dropout_rate (float): The dropout rate to prevent overfitting.
        learning_rate (float): The learning rate for optimization.

    Returns:
        Sequential: A Keras Sequential model representing the CNN-LSTM architecture.
    """
    model = Sequential()
    
    # Add a 1D convolutional layer with specified filters and kernel size
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    
    # Add MaxPooling1D layer only if the input is large enough
    if input_shape[0] > kernel_size:
        model.add(MaxPooling1D(pool_size=2))
    
    # Add a Long Short-Term Memory (LSTM) layer with specified units and activation function
    model.add(LSTM(lstm_units, activation='tanh'))
    
    # Flatten the output of LSTM layer
    model.add(Flatten())
    
    # Add a fully connected dense layer with specified units and ReLU activation
    model.add(Dense(dense_units, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    
    # Add dropout layer to prevent overfitting
    model.add(Dropout(dropout_rate))
    
    # Use 'softmax' activation for multi-class classification and 'sigmoid' for binary classification
    model.add(Dense(num_classes, activation='softmax' if num_classes > 1 else 'sigmoid'))
    
    # Compile the model with the specified optimizer, loss function, and metrics
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
def lr_schedule(epoch, lr):
    """
    Learning rate schedule function to adjust learning rate during training.

    Args:
        epoch (int): The current epoch number.
        lr (float): The current learning rate.

    Returns:
        float: The updated learning rate for the epoch.
    """
    if epoch < 10:
        return lr  # Use initial learning rate for the first 10 epochs
    else:
        return lr * tf.math.exp(-0.1)  # Adjust learning rate exponentially

In [ ]:
def tune_hyperparameters(input_shape, num_classes, X_train, y_train, X_val, y_val):
    """
    Tune hyperparameters for a CNN-LSTM model using Keras Tuner.

    Args:
        input_shape (tuple): The shape of the input data (e.g., (sequence_length, input_features)).
        num_classes (int): The number of output classes.
        X_train (numpy.ndarray): Training data.
        y_train (numpy.ndarray): Training labels.
        X_val (numpy.ndarray): Validation data.
        y_val (numpy.ndarray): Validation labels.

    Returns:
        dict: The best hyperparameters found during tuning.
    """
    def model_builder(hp):
        """
        Build a CNN-LSTM model with hyperparameters specified by the Keras Tuner.

        Args:
            hp (HyperParameters): An instance of the Keras Tuner HyperParameters class used to define hyperparameters.

        Returns:
            Sequential: A Keras Sequential model representing the CNN-LSTM architecture with hyperparameter configurations.
        """
        # Define hyperparameters to be tuned
        hp_filters = hp.Int('filters', min_value=16, max_value=64, step=16)
        hp_kernel_size = hp.Choice('kernel_size', values=[3, 5])
        hp_lstm_units = hp.Int('lstm_units', min_value=30, max_value=100, step=10)
        hp_dense_units = hp.Int('dense_units', min_value=16, max_value=64, step=16)
        hp_dropout_rate = hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
        hp_learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')

        # Build the model with the specified hyperparameters
        model = create_lstm_cnn_model(input_shape, num_classes, hp_filters, hp_kernel_size, hp_lstm_units, hp_dense_units, hp_dropout_rate, hp_learning_rate)

        # Use Adam optimizer with a custom learning rate
        optimizer = Adam(learning_rate=hp_learning_rate)

        # Compile the model with the custom learning rate scheduler
        model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        return model

    # Initialize a Hyperband tuner
    tuner = Hyperband(
        model_builder,
        objective='val_accuracy',
        max_epochs=50,
        factor=5,
        directory='my_dir',
        project_name='lstm_cnn_tuning_hyperband')

    # Define a grid search over specified hyperparameters
    hp = HyperParameters()
    hp.Int('filters', min_value=16, max_value=64, step=16)
    hp.Choice('kernel_size', values=[3, 5])
    hp.Int('lstm_units', min_value=30, max_value=100, step=10)
    hp.Int('dense_units', min_value=16, max_value=64, step=16)
    hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)
    hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')

    # Define a learning rate scheduler callback
    lr_scheduler = LearningRateScheduler(lr_schedule)

    # Start hyperparameter tuning
    tuner.search(X_train, y_train, epochs=50, validation_data=(X_val, y_val), callbacks=[lr_scheduler], hyperparameters=hp)

    # Get the best hyperparameters found during tuning
    best_hyperparameters = tuner.get_best_hyperparameters()[0]

    return best_hyperparameters

In [ ]:
def cross_validate_model(X, y, input_shape, num_classes, best_hps, n_folds=4, n_repeats=1, epochs=50, batch_size=32):
    """
    Perform cross-validation for a CNN-LSTM model.

    Args:
        X (numpy.ndarray): Input data.
        y (numpy.ndarray): Target labels.
        input_shape (tuple): The shape of the input data (e.g., (sequence_length, input_features)).
        num_classes (int): The number of output classes.
        best_hps (dict): A dictionary containing the best hyperparameters found during tuning.
        n_folds (int, optional): The number of folds for cross-validation. Default is 4.
        n_repeats (int, optional): The number of times to repeat cross-validation. Default is 1.
        epochs (int, optional): The number of training epochs. Default is 50.
        batch_size (int, optional): The batch size for training. Default is 32.

    Returns:
        float: The average accuracy across all cross-validation repeats.
    """
    # Initialize results list
    all_results = []

    # Repeated cross-validation
    for _ in range(n_repeats):
        # Define K-fold cross-validator
        kf = KFold(n_splits=n_folds, shuffle=True, random_state=None)
        
        # Initialize results for this repeat
        results = []

        # K-fold cross-validation
        for train_index, val_index in kf.split(X):
            # Split data
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]

            # Create and compile the model with the best hyperparameters
            model = create_lstm_cnn_model(
                input_shape=input_shape, 
                num_classes=num_classes, 
                filters=best_hps.get('filters'), 
                kernel_size=best_hps.get('kernel_size'), 
                lstm_units=best_hps.get('lstm_units'), 
                dense_units=best_hps.get('dense_units'), 
                dropout_rate=best_hps.get('dropout_rate'), 
                learning_rate=best_hps.get('learning_rate'))

            # EarlyStopping callback
            early_stopping_callback = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=5, restore_best_weights=True)

            # Fit the model
            model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, batch_size=batch_size, callbacks=[early_stopping_callback], verbose=0)

            # Evaluate the model
            loss, accuracy = model.evaluate(X_val, y_val)
            results.append(accuracy)

        # Calculate average performance for this repeat
        average_performance = np.mean(results)
        all_results.append(average_performance)

    # Calculate average performance across all repeats
    overall_average_performance = np.mean(all_results)

    return overall_average_performance

In [ ]:
def evaluate_model(model, X_test, y_test, run_results):
    """
    Evaluate a trained model on test data and collect performance metrics.

    Args:
        model: A trained Keras model.
        X_test (numpy.ndarray): Test input data.
        y_test (numpy.ndarray): Test target labels.
        run_results (dict): A dictionary to store evaluation results.
    """
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1) if y_pred.shape[1] > 1 else (y_pred > 0.5).astype('int32')
    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred_classes)
    run_results['accuracy'] = accuracy
    # Classification report
    report = classification_report(y_test, y_pred_classes, output_dict=True)
    # Extract the required metrics using .get() for safer access
    weighted_avg = report.get('weighted avg', {})
    run_results['precision'] = weighted_avg.get('precision', None)
    run_results['recall'] = weighted_avg.get('recall', None)
    run_results['f1-score'] = weighted_avg.get('f1-score', None)
    # Check if the values were successfully retrieved
    if run_results['precision'] is None or run_results['recall'] is None or run_results['f1-score'] is None:
        print("Some metrics were not found in the classification report.")

In [ ]:
def train_model(X_train, y_train, X_val, y_val, input_shape, num_classes, best_hps, num_runs):
    """
    Train a CNN-LSTM model with hyperparameters and evaluate its performance.

    Args:
        X_train (numpy.ndarray): Training input data.
        y_train (numpy.ndarray): Training target labels.
        X_val (numpy.ndarray): Validation input data.
        y_val (numpy.ndarray): Validation target labels.
        input_shape (tuple): The shape of the input data (e.g., (sequence_length, input_features)).
        num_classes (int): The number of output classes.
        best_hps (dict): A dictionary containing the best hyperparameters found during tuning.
        num_runs (int): The number of training runs to perform.

    Returns:
        tuple: A tuple containing the trained model and its training history.
    """
    # Default values for hyperparameters
    DEFAULT_EPOCHS = 50
    DEFAULT_BATCH_SIZE = 32
    DEFAULT_LEARNING_RATE = 0.001
    DEFAULT_FILTERS = 64
    DEFAULT_KERNEL_SIZE = 3
    DEFAULT_LSTM_UNITS = 100
    DEFAULT_DENSE_UNITS = 100
    DEFAULT_DROPOUT_RATE = 0.5

    average_results = []
    for run in range(num_runs):
        run_results = {}

        # Extract hyperparameters safely with default values
        filters = best_hps.get('filters') if 'filters' in best_hps else DEFAULT_FILTERS
        kernel_size = best_hps.get('kernel_size') if 'kernel_size' in best_hps else DEFAULT_KERNEL_SIZE
        lstm_units = best_hps.get('lstm_units') if 'lstm_units' in best_hps else DEFAULT_LSTM_UNITS
        dense_units = best_hps.get('dense_units') if 'dense_units' in best_hps else DEFAULT_DENSE_UNITS
        dropout_rate = best_hps.get('dropout_rate') if 'dropout_rate' in best_hps else DEFAULT_DROPOUT_RATE
        learning_rate = best_hps.get('learning_rate') if 'learning_rate' in best_hps else DEFAULT_LEARNING_RATE
        epochs = best_hps.get('epochs') if 'epochs' in best_hps else DEFAULT_EPOCHS
        batch_size = best_hps.get('batch_size') if 'batch_size' in best_hps else DEFAULT_BATCH_SIZE

        # Create the model with the hyperparameters
        model = create_lstm_cnn_model(input_shape, num_classes, filters, kernel_size, lstm_units, dense_units, dropout_rate, learning_rate)

        # Early stopping callback
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Train the model with early stopping
        history = model.fit(X_train, y_train, validation_data=(X_val, y_val),
                            epochs=epochs, batch_size=batch_size, callbacks=[early_stopping], verbose=0)

        # After training, evaluate the model and collect results
        evaluate_model(model, X_val, y_val, run_results)
        
        # Print the classification report
        print(f"Classification Report (Run {run + 1}):")
        print(classification_report(y_val, model.predict(X_val) > 0.5))  # Adjust the threshold if needed

        # Append the results of this run to the average_results list
        average_results.append(run_results)

    # Calculate and print the average results across all runs
    average_precision = sum(result['precision'] for result in average_results) / len(average_results)
    average_recall = sum(result['recall'] for result in average_results) / len(average_results)
    average_f1_score = sum(result['f1-score'] for result in average_results) / len(average_results)
    average_accuracy = sum(result.get('accuracy', 0) for result in average_results) / len(average_results)

    # Print average results
    print(f'Average Precision: {average_precision:.2f}')
    print(f'Average Recall: {average_recall:.2f}')
    print(f'Average F1-Score: {average_f1_score:.2f}')
    print(f'Average Accuracy: {average_accuracy:.2f}')

    return model, history

In [ ]:
def main():
    print("Loading the dataset...")
    df = load_data("COVID-Full.csv")
    
    print("Identifying data types...")
    numerical_cols, categorical_cols = identify_data_types(df)
    
    print("Performing EDA...")
    exploratory_data_analysis(df, numerical_cols, categorical_cols)
    
    print("Normalizing and encoding data...")
    df_encoded = normalize_and_encode(df, numerical_cols, categorical_cols)
    
    print("Handling missing values...")
    df_no_missing = handle_missing_values(df_encoded)
    df_no_missing = add_missing_columns(df_no_missing, df)
        
    print("Performing feature selection...")
    df_selected, selected_features = feature_selection(df_no_missing, 'ICU')
    
    print("Preparing sequence data...")
    X, y = prepare_sequence_data(df_selected, 'PATIENT_VISIT_IDENTIFIER', 'ICU', 'WINDOW')
    
    print("Converting data to numpy arrays...")
    X_array = np.array(X, dtype=object)
    y_array = np.array(y)
    
    print("Padding sequences...")
    max_sequence_length = max(len(sequence) for sequence in X_array)
    feature_size = X_array[0].shape[1]
    X_padded = np.array([np.pad(sequence, ((0, max_sequence_length - len(sequence)), (0, 0)), mode='constant', constant_values=0) for sequence in X_array])
    
    print("Splitting data into training and validation sets...")
    X_train, X_val, y_train, y_val = train_test_split(X_padded, y_array, test_size=0.2, random_state=42)
    
    print("Determining model parameters...")
    input_shape = (max_sequence_length, feature_size)
    num_classes = 1
    
    print("Performing hyperparameter tuning...")
    best_hps = tune_hyperparameters(input_shape, num_classes, X_train, y_train, X_val, y_val)
    
    print("Performing K-Fold Cross-Validation...")
    average_accuracy = cross_validate_model(X_padded, y_array, input_shape, num_classes, best_hps, n_folds=4, epochs=50, batch_size=32)
    
    print(f"Average Accuracy across folds: {average_accuracy}")
    
    print("Training the model with best hyperparameters...")
    model, history = train_model(X_train, y_train, X_val, y_val, input_shape, num_classes, best_hps, num_runs=10)

<h1>Script Start</h1>

In [ ]:
if __name__ == "__main__":
    main()